<a href="https://colab.research.google.com/github/mananparikh1307/bigdata/blob/main/YAFIMTIME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet pyspark

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("apriori").setMaster("local")
sc = SparkContext(conf=conf)

ValueError: ignored

In [ ]:
file = "/content/gdrive/My Drive/DoubleSize.csv"
lineRDD = sc.textFile(file)
RDD = lineRDD.map(lambda line : line.split(","))

In [ ]:
import pandas as pd
import timeit
tr = pd.read_csv(file)
tr.columns

Index(['bruises', 'gill-attachment', 'gill-close-spacing', 'gill-narrow-size',
       'enlarge-stalk-shape', 'poisonous', 'edible', 'convex_cap', 'bell_cap',
       'sunken_cap',
       ...
       'several_pop', 'solitary_pop', 'clustered_pop', 'urban', 'grasses',
       'meadows', 'woods', 'paths', 'waste', 'leaves'],
      dtype='object', length=113)

In [ ]:
def Count(rdd):
  count = 0
  for i in rdd:
    if( i[0] == '1' ):
      count = count + 1

  return count

In [ ]:
from collections import OrderedDict
import numpy as np

L_1 = {}
min_sup = 8124
for i in range(len(tr.columns)):
  rdd = RDD.map(lambda line: (line[i],1))
  count = Count(rdd.collect())
  if( count >= min_sup ):
    L_1[tr.columns[i]] = count
L_1 = (sorted(L_1.items()))
L_1

[('edible', 8416),
 ('gill-close-spacing', 13624),
 ('one_ring', 14976),
 ('partial_veil', 16248),
 ('smooth_above_ring', 10352),
 ('smooth_below_ring', 9872),
 ('white_above_ring', 8928),
 ('white_below_ring', 8768),
 ('white_veil', 15848)]

In [ ]:
def Candidate_Set(k,list):
  Cand = []
  for i in list:
    for j in list:
      temp = True
      if( k > 2 and i[0] != j[0] ):
        for l in range(k-2):
          if(i[0][l] != j[0][l]):
            temp = False
            break
        if( temp == True ):
          l = []
          for a in range(k-1):
            l.append(i[0][a])
          l.append(j[0][k-2])
          Cand.append(sorted(l))
      else:
        l = []
        if( i[0] != j[0] ):
          l.append(i[0])
          l.append(j[0])
          Cand.append(sorted(l))
  Cand = sorted(Cand)
  res = [t for t in (set(tuple(i) for i in Cand))]
  res = sorted(res)
  return res

In [ ]:
def Candidate_Pruning(RDD,list,k):
  can_pru = {}

  for j in list:
    can_pru[j] = 0

  for i in RDD:
    for j in list:
      count = 0
      for l in range(k):
        index = tr.columns.get_loc(j[l])
        if( i[index] == '1' ):
          count = count + 1
      if( count == k ):
        can_pru[j] = can_pru[j] + 1

  pruned = {}
  for i in can_pru.keys():
    if( can_pru[i] != 0 and can_pru[i] >= min_sup ):
      pruned[i] = can_pru[i]

  pruned = sorted(pruned.items())
  return pruned

In [ ]:
Freq_List = [L_1]
k = 2

candidate_set = Candidate_Set(k,L_1)
candidate_pruned = Candidate_Pruning(RDD.collect(), candidate_set, k)
Freq_List.append(candidate_pruned)
k = k + 1

while(True):
  candidate_set = Candidate_Set(k,candidate_pruned)
  candidate_pruned = Candidate_Pruning(RDD.collect(),candidate_set, k)
  if( len(candidate_pruned) == 0 ):
    break
  else:
    Freq_List.append(candidate_pruned)
    k = k + 1

In [ ]:
print(k)

5


In [ ]:
start = timeit.default_timer()
Freq_List
stop = timeit.default_timer()

print('Time: ', stop - start) 

Time:  0.0001601510011823848


In [ ]:
start = timeit.default_timer()
can2 = Candidate_Set(2,L_1)
res2 = Candidate_Pruning(RDD.collect(), can2, 2)
can2
stop = timeit.default_timer()

print('Time: ', stop - start) 

Time:  2.3230930809986603


In [ ]:
start = timeit.default_timer()
can3 = Candidate_Set(3,res2)
res3 = Candidate_Pruning(RDD.collect(), can3, 3)
res3
stop = timeit.default_timer()

print('Time: ', stop - start) 

Time:  2.3023290490000363


In [ ]:
start = timeit.default_timer()
can4 = Candidate_Set(4,res3)
res4 = Candidate_Pruning(RDD.collect(), can4, 4)
res4
stop = timeit.default_timer()

print('Time: ', stop - start) 

Time:  1.3386787330000516


In [ ]:
start = timeit.default_timer()
can5 = Candidate_Set(5,res4)
res5 = Candidate_Pruning(RDD.collect(), can5, 5)
res5
stop = timeit.default_timer()

print('Time: ', stop - start) 

Time:  0.4246114130000933


In [ ]:
start = timeit.default_timer()
can6 = Candidate_Set(6,res5)
res6 = Candidate_Pruning(RDD.collect(), can6, 6)
res6
stop = timeit.default_timer()

print('Time: ', stop - start) 

Time:  0.21982010799911222


In [ ]:
start = timeit.default_timer()
can7 = Candidate_Set(7,res6)
res7 = Candidate_Pruning(RDD.collect(), can7, 7)
res7
stop = timeit.default_timer()

print('Time: ', stop - start) 

Time:  0.281091741999262


In [ ]:
start = timeit.default_timer()
can8 = Candidate_Set(8,res7)
res8 = Candidate_Pruning(RDD.collect(), can8, 8)
res8
stop = timeit.default_timer()

print('Time: ', stop - start) 

Time:  0.12505043999954069


In [ ]:
start = timeit.default_timer()
can9 = Candidate_Set(9,res8)
res9 = Candidate_Pruning(RDD.collect(), can9, 9)
res9
stop = timeit.default_timer()

print('Time: ', stop - start) 

Time:  0.2208671719999984
